# 7. 數據規整化 - 清理、轉換、合併、重塑#

In [2]:
%pylab
from pandas import Series, DataFrame
import pandas as pd
import re

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


## 合併數據集##

### 數據庫風格的DataFrame合併###

In [45]:
# pandas 的 merge() 方法
df1 = DataFrame({'key': list('bbacaabd'), 
                 'data1': range(8)})
df2 = DataFrame({'key': list('abd'), 
                 'data2': range(3)})

In [46]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b
7,7,d


In [47]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [48]:
# 用 merge() 方法，預設以共同的column 'key' 來 join 兩個 DataFrame
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0
6,7,d,2


In [49]:
# 也可以顯式的指定 要以哪一個欄位來 join
pd.merge(df1, df2, on = 'key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0
6,7,d,2


In [50]:
# 如果兩個物件的列名不同，也可以分別指定
df3 = DataFrame({'lkey': list('bbacaabd'), 
                 'data1': range(8)})
df4 = DataFrame({'rkey': list('abd'), 
                 'data2': range(3)})

In [51]:
df3

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b
7,7,d


In [52]:
df4

,data2,rkey
0,0,a
1,1,b
2,2,d


In [53]:
# 分別指定各要以哪一個欄位來join
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey')
# lkey = 'c' 的項目不會出現，因為 merge()方法預設以 inner join 的模式來merge

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a
6,7,d,2,d


In [54]:
# 可以以 how 參數指定 join的模式 (outer 聯集)
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey', how = 'outer')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a
6,3,c,NaN,NaN
7,7,d,2,d


In [55]:
# 可以以 how 參數指定 join的模式 (inner)
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey', how = 'inner')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a
6,7,d,2,d


In [56]:
# 可以以 how 參數指定 join的模式 (left)
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey', how = 'left')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,2,a,0,a
3,3,c,NaN,NaN
4,4,a,0,a
5,5,a,0,a
6,6,b,1,b
7,7,d,2,d


In [57]:
# 可以以 how 參數指定 join的模式 (right)
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey', how = 'right')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a
6,7,d,2,d


In [58]:
# 多對多的合併
df1 = DataFrame({'key': list('bbacab'), 
                'data1': range(6)})
df2 = DataFrame({'key': list('ababd'), 
                'data2': range(5)})

In [59]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [60]:
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [61]:
# 因為 df2中 key a, b 都有對應多個值，所以會產生多個對應 rows
pd.merge(df1, df2, on = 'key', how = 'left')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,2,a,0
5,2,a,2
6,3,c,NaN
7,4,a,0
8,4,a,2
9,5,b,1


In [62]:
# 因為 df2中 key a, b 都有對應多個值，所以會產生多個對應 rows
pd.merge(df1, df2, on = 'key', how = 'inner')
# inner join 是求交集，所以不會有 NaN的值出現

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [63]:
# 可以根據多個 keys來 join
df1 = DataFrame({'key1': ['foo', 'foo', 'bar'], 
                 'key2': ['one', 'two', 'one'], 
                 'data': [1, 2, 3]})
df2 = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 
                 'key2': ['one', 'one', 'one', 'two'], 
                 'data': [4, 5, 6, 7]})

In [64]:
df1

,data,key1,key2
0,1,foo,one
1,2,foo,two
2,3,bar,one


In [65]:
df2

,data,key1,key2
0,4,foo,one
1,5,foo,one
2,6,bar,one
3,7,bar,two


In [66]:
# 可以根據多個 keys來 join
pd.merge(df1, df2, on = ['key1', 'key2'], how = 'outer')

,data_x,key1,key2,data_y
0,1,foo,one,4
1,1,foo,one,5
2,2,foo,two,NaN
3,3,bar,one,6
4,NaN,bar,two,7


In [67]:
# 如果列名重複 ('key2')
pd.merge(df1, df2, on = ['key1'], how = 'outer')

,data_x,key1,key2_x,data_y,key2_y
0,1,foo,one,4,one
1,1,foo,one,5,one
2,2,foo,two,4,one
3,2,foo,two,5,one
4,3,bar,one,6,one
5,3,bar,one,7,two


In [68]:
# 如果列名重複 ('key2')
# 使用 suffix 參數來指定附加到左右兩個 DataFrame重複列名的名稱上
pd.merge(df1, df2, on = ['key1'], how = 'outer', suffixes = ['_left', '_right'])

,data_left,key1,key2_left,data_right,key2_right
0,1,foo,one,4,one
1,1,foo,one,5,one
2,2,foo,two,4,one
3,2,foo,two,5,one
4,3,bar,one,6,one
5,3,bar,one,7,two


In [69]:
# 設定 sort 參數， 依據 keys來排序
pd.merge(df1, df2, on = ['key1', 'key2'], how = 'outer', suffixes = ['_left', '_right'], sort = True)

,data_left,key1,key2,data_right
0,3,bar,one,6
1,NaN,bar,two,7
2,1,foo,one,4
3,1,foo,one,5
4,2,foo,two,NaN


### 索引上的合併###

In [70]:
# 使用DataFrame的索引作為 join的 key
dfl = DataFrame({'key': list('abaabc'), 
                 'value': range(6)})
# dfr 以 ['a', 'b'] 作為索引
dfr = DataFrame({'value': [3.5, 7]}, index = list('ab'))

In [71]:
dfl

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [72]:
# dfr 以 ['a', 'b'] 作為索引
dfr

,value
a,3.5
b,7.0


In [73]:
# 設定 right_index = True，表示 right DataFrame使用索引作為 join 的 key欄位
pd.merge(dfl, dfr, left_on = 'key', right_index = True, how = 'outer', suffixes = ['_left', '_right'], sort = True)

,key,value_left,value_right
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [74]:
# 階層化索引
dfl = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 
                 'key2': [2000, 2001, 2002, 2001, 2002], 
                 'data': np.arange(5.)})
dfr = DataFrame(np.arange(12).reshape((6, 2)), 
                columns = ['data1', 'data2'],
                index = [['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                         [2001, 2000, 2000, 2000, 2001, 2002]])
dfr.index.names = ['state', 'year']

In [75]:
dfl

,data,key1,key2
0,0,Ohio,2000
1,1,Ohio,2001
2,2,Ohio,2002
3,3,Nevada,2001
4,4,Nevada,2002


In [76]:
dfr

data1  data2
state  year              
Nevada 2001      0      1
       2000      2      3
Ohio   2000      4      5
       2000      6      7
       2001      8      9
       2002     10     11

In [77]:
# 階層化索引
# 左方指定用來join的 keys:  left_on = ['key1', 'key2']
# 右方指定使用索引來做為 join的 keys: right_index = True
pd.merge(dfl, dfr, left_on = ['key1', 'key2'], right_index = True, how = 'outer', suffixes = ['_left', '_right'], sort = True)

,data,key1,key2,data1,data2
4,NaN,Nevada,2000,2,3
3,3,Nevada,2001,0,1
4,4,Nevada,2002,NaN,NaN
0,0,Ohio,2000,4,5
0,0,Ohio,2000,6,7
1,1,Ohio,2001,8,9
2,2,Ohio,2002,10,11


In [78]:
# 同時使用合併雙方的索引
dfl = DataFrame([[1., 2.], [3., 4.], [5., 6.]], 
                index = list('ace'), 
                columns = ['Ohio', 'Nevada'])
dfr = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14]],
                index = list('bcde'), 
                columns = ['Missouri', 'Alabama'])

In [79]:
dfl

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [80]:
dfr

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [81]:
pd.merge(dfl, dfr, left_index = True, right_index = True, how = 'outer', suffixes = ['_left', '_right'], sort = True )

,Ohio,Nevada,Missouri,Alabama
a,1,2,NaN,NaN
b,NaN,NaN,7,8
c,3,4,9,10
d,NaN,NaN,11,12
e,5,6,13,14


In [82]:
# DataFrame物件有一個實例方法 join()，可以用來合併多個帶有相同或相似索引的DataFrame物件，不論他們之間是否有重疊的列
dfl.join(dfr, how = 'outer', sort = True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,NaN,NaN
b,NaN,NaN,7,8
c,3,4,9,10
d,NaN,NaN,11,12
e,5,6,13,14


In [83]:
df2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14]],
                index = list('bcde'), 
                columns = ['New York', 'Miami'])

# join()方法可以接受 DataFrame 序列，內含有多個 DataFrame物件，作為 join的對象
dfl.join([dfr, df2], how = 'outer', sort = True)

,Ohio,Nevada,Missouri,Alabama,New York,Miami
a,1,2,NaN,NaN,NaN,NaN
b,NaN,NaN,7,8,7,8
c,3,4,9,10,9,10
d,NaN,NaN,11,12,11,12
e,5,6,13,14,13,14


### 軸向連接(concatenation)###

In [84]:
# NumPy中有 concatenate()方法
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [85]:
# NumPy 的 concatenate()方法
np.concatenate([arr, arr], axis = 1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [86]:
# pandas 有 concat()方法
s1 = Series([0, 1], index = ['a', 'b'])
s2 = Series([2, 3, 4], index = ['c', 'd', 'e'])
s3 = Series([5, 6], index = ['f', 'g'])
sc = pd.concat([s1, s2, s3])
sc

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [87]:
type(sc)

pandas.core.series.Series

In [88]:
# concat()方法預設以 axis = 0 來連接，如果傳入 axis = 1，則會產生一個 DataFrame
sc = pd.concat([s1, s2, s3], axis = 1)
sc

,0,1,2
a,0,NaN,NaN
b,1,NaN,NaN
c,NaN,2,NaN
d,NaN,3,NaN
e,NaN,4,NaN
f,NaN,NaN,5
g,NaN,NaN,6


In [89]:
type(sc)

pandas.core.frame.DataFrame

In [90]:
# 傳入 join = 'inner' 可以看到交集
s4 = pd.concat([s1 * 5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [91]:
s1

a    0
b    1
dtype: int64

In [92]:
pd.concat([s1, s4], axis = 1)

,0,1
a,0,0
b,1,5
f,NaN,5
g,NaN,6


In [93]:
# 傳入 join = 'inner' 可以看到交集
sc = pd.concat([s1, s4], axis = 1, join = 'inner')
sc

,0,1
a,0,0
b,1,5


In [94]:
# 透過 join_axes 參數，指定要在其他軸上使用的索引
pd.concat([s1, s4], axis = 1, join_axes = [['a', 'c', 'b', 'e']])

,0,1
a,0,0
c,NaN,NaN
b,1,5
e,NaN,NaN


In [95]:
s3

f    5
g    6
dtype: int64

In [96]:
# 使用 keys 參數，建立 階層式索引
result = pd.concat([s1, s1, s3], keys = ['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [97]:
# 把具有層次化索引的 Series， unstack 成為 DataFrame
result.unstack()

,a,b,f,g
one,0,1,NaN,NaN
two,0,1,NaN,NaN
three,NaN,NaN,5,6


In [98]:
# 沿著 axis = 1 做 concat，keys就會成為 列頭
result = pd.concat([s1, s1, s3], axis = 1, keys = ['one', 'two', 'three'])
result

,one,two,three
a,0,0,NaN
b,1,1,NaN
f,NaN,NaN,5
g,NaN,NaN,6


In [99]:
# 同樣的邏輯對 DataFrame也是一樣的
# 沿著 axis = 1 做 concat，keys就會成為 列頭
df1 = DataFrame(np.arange(6).reshape((3, 2)), 
                index = ['a', 'b', 'c'], 
                columns = ['one', 'two'])
df2 = DataFrame(5 + np.arange(4).reshape((2, 2)), 
                index = ['a', 'c'], 
                columns = ['three', 'four'])
pd.concat([df1, df2], axis = 1, keys = ['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1      5    6
b      2   3    NaN  NaN
c      4   5      7    8

In [100]:
# 傳入一個字典，則字典的鍵就會被當作keys參數的值
# 這種表達方式比較容易讀懂
pd.concat({'level1': df1, 'level2': df2}, axis = 1)

level1     level2     
     one two  three four
a      0   1      5    6
b      2   3    NaN  NaN
c      4   5      7    8

In [101]:
# names 參數，設定層次化所引的名稱
pd.concat({'level1': df1, 'level2': df2}, axis = 1, names = ['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1      5    6
b          2   3    NaN  NaN
c          4   5      7    8

In [102]:
# 和當下分析工作無關的row索引
df1 = DataFrame(np.random.randn(3, 4), columns = list('abcd'))
df2 = DataFrame(np.random.randn(2, 3), columns = list('bda'))

In [103]:
df1

,a,b,c,d
0,0.708228,-1.584397,0.252971,-1.307792
1,0.506975,-1.075091,-0.723926,-1.437577
2,1.260511,0.285505,-1.227035,-1.238654


In [104]:
df2

,b,d,a
0,2.05225,0.232492,1.081393
1,-0.78755,-1.179561,-2.246684


In [105]:
# concat之後，會保留原來的索引
pd.concat([df1, df2])

,a,b,c,d
0,0.708228,-1.584397,0.252971,-1.307792
1,0.506975,-1.075091,-0.723926,-1.437577
2,1.260511,0.285505,-1.227035,-1.238654
0,1.081393,2.052250,NaN,0.232492
1,-2.246684,-0.787550,NaN,-1.179561


In [106]:
# ignore_index = True，不保留原本的索引
pd.concat([df1, df2], ignore_index = True)

,a,b,c,d
0,0.708228,-1.584397,0.252971,-1.307792
1,0.506975,-1.075091,-0.723926,-1.437577
2,1.260511,0.285505,-1.227035,-1.238654
3,1.081393,2.052250,NaN,0.232492
4,-2.246684,-0.787550,NaN,-1.179561


### 合併重疊數據###

In [107]:
# NumPy的 where 函數
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], 
           index = list('abcdef'))
b = Series(np.arange(len(a)), dtype = np.float64, 
           index = list('abcdef')) 

In [108]:
a

a    NaN
b    2.5
c    NaN
d    3.5
e    4.5
f    NaN
dtype: float64

In [109]:
b[-1] = np.nan

In [110]:
b

a     0
b     1
c     2
d     3
e     4
f   NaN
dtype: float64

In [111]:
# NumPy的 where 函數，是一種向量化的 if-else
np.where(pd.isnull(a), b, a)

array([ 0. ,  2.5,  2. ,  3.5,  4.5,  nan])

In [112]:
b[:-2]

a    0
b    1
c    2
d    3
dtype: float64

In [113]:
a[2:]

c    NaN
d    3.5
e    4.5
f    NaN
dtype: float64

In [114]:
# Series 的 combine_first()方法，也是一樣的功能，且會自動對齊數據
b[:-2].combine_first(a[2:])

a    0.0
b    1.0
c    2.0
d    3.0
e    4.5
f    NaN
dtype: float64

In [115]:
# 對於DataFrame，combine_first的功能就像是在對缺失數據 打補釘
df1 = DataFrame({'a': [1., np.nan, 5., np.nan], 
                 'b': [np.nan, 2., np.nan, 5.], 
                 'c': list(range(2, 18, 4))})
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.], 
                 'b': [np.nan, 3., 4., 6., 8.]})

In [116]:
df1

,a,b,c
0,1,NaN,2
1,NaN,2,6
2,5,NaN,10
3,NaN,5,14


In [117]:
df2

,a,b
0,5,NaN
1,4,3
2,NaN,4
3,3,6
4,7,8


In [118]:
# 對於 df1中的缺失數據，會嘗試以df2中的對應數據補充
df1.combine_first(df2)

,a,b,c
0,1,NaN,2
1,4,2,6
2,5,4,10
3,3,5,14
4,7,8,NaN


## 重塑(reshape)和軸向旋轉(pivot)##

### 重塑層次化索引###

In [119]:
# 主要兩種方法
# stack: 將 column 旋轉為 row
# unstack: 將 row 旋轉為 column

In [120]:
df = DataFrame(np.arange(6).reshape((2, 3)), 
               index = pd.Index(['Ohio', 'Colorado'], name = 'state'), 
               columns = pd.Index(['one', 'two', 'three'], name = 'number'))
df               

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [121]:
# stack: 將 column 旋轉為 row
s = df.stack()
s

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [122]:
# s 是一個 Series 物件 
type(s)

pandas.core.series.Series

In [123]:
# unstack: 將 row 旋轉為 column
# Series會變成一個 DataFrame
s.unstack()

D:\Python\lib\site-packages\pandas\core\index.py:4281: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.sum(name == np.asarray(self.names)) > 1


number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [124]:
# 默認情況下，stack, unstack 操作的是最內層
# 可以傳入分層級別的編號或者名噌，以對其他級別操作
s.unstack(0)

D:\Python\lib\site-packages\pandas\core\index.py:4281: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return np.sum(name == np.asarray(self.names)) > 1


state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [125]:
s.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [126]:
# 如果不是所有的級別值都可以在分組中找到的話，則unstack操作可以會產生缺失數據
s1 = Series([0, 1, 2, 3], index = list('abcd'))
s2 = Series([4, 5, 6], index = list('cde'))
data2 = pd.concat([s1, s2], keys = ['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [127]:
# unstack操作可以會產生缺失數據
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,NaN
two,NaN,NaN,4,5,6


In [128]:
# stack 預設會濾除缺失數據，因此 stack/unstack 是可逆的
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: float64

In [129]:
# 也可以設定 dropna 參數，不要濾除缺失數據
data2.unstack().stack(dropna = False)

one  a     0
     b     1
     c     2
     d     3
     e   NaN
two  a   NaN
     b   NaN
     c     4
     d     5
     e     6
dtype: float64

In [130]:
# unstack操作中，旋轉軸的級別將會成為結果中的最低級別
df = DataFrame({'left': s, 'right': s + 5}, 
               columns = pd.Index(['left', 'right'], name = 'side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [131]:
# 索引'state'經過unstack之後，成為最內層的 column索引
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [132]:
df.unstack('state').unstack('side')

side   state     number
left   Ohio      one        0
                 two        1
                 three      2
       Colorado  one        3
                 two        4
                 three      5
right  Ohio      one        5
                 two        6
                 three      7
       Colorado  one        8
                 two        9
                 three     10
dtype: int32

In [133]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [134]:
df.unstack('number').unstack('state')

side   number  state   
left   one     Ohio         0
               Colorado     3
       two     Ohio         1
               Colorado     4
       three   Ohio         2
               Colorado     5
right  one     Ohio         5
               Colorado     8
       two     Ohio         6
               Colorado     9
       three   Ohio         7
               Colorado    10
dtype: int32

### 將"長格式"旋轉為"寬格式"###

In [136]:
# ldata = pd.read_csv(open('Book1.csv'))
# ldata
# 長格式 
# 好處: 值的種類可以隨時增加或減少
# 缺點: 操作起來較麻煩，不易閱讀

In [137]:
# ldata_string = ldata.to_json()
# ldata_string

In [138]:
# 重新設定 ldata_string，不用依靠檔案載入
ldata_string = """
{"date":{"0":"1959\\/3\\/31","1":"1959\\/3\\/31","2":"1959\\/3\\/31","3":"1959\\/6\\/30","4":"1959\\/6\\/30","5":"1959\\/6\\/30","6":"1959\\/9\\/30","7":"1959\\/9\\/30","8":"1959\\/9\\/30"},"item":{"0":"realgdp","1":"infl","2":"unemp","3":"realgdp","4":"infl","5":"unemp","6":"realgdp","7":"infl","8":"unemp"},"value":{"0":2710.349,"1":0.0,"2":5.8,"3":2712.349,"4":2.0,"5":7.8,"6":2714.349,"7":4.0,"8":9.8}}
"""

In [139]:
df = DataFrame(json.loads(ldata_string))
df
# 長格式 
# 好處: 值的種類可以隨時增加或減少
# 缺點: 操作起來較麻煩，不易閱讀

NameError: name 'json' is not defined

In [ ]:
# pivot()方法 可以將 長格式 轉換為 寬格式
# 總共需要 index, columns, values 三個參數
pivoted = ldata.pivot(index = 'date', columns = 'item', values = 'value')
pivoted

In [ ]:
# 增加一列 value2
ldata['value2'] = np.random.randn(len(ldata))
ldata

In [ ]:
# 如果只指定 index, columns，則DataFrame就會具有層次化的columns
pivoted = ldata.pivot(index = 'date', columns = 'item')
pivoted

In [ ]:
pivoted['value'][:3]

In [ ]:
# 也可以用 set_index建立層次化的索引，然後再用 unstack建置
ldata.set_index(['date', 'item'])

In [ ]:
ldata.set_index(['date', 'item']).unstack('item')

## 數據轉換##

### 移除重複數據###

In [140]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4, 
                  'k2': [1, 1, 2, 3, 3, 4, 4,]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [141]:
# DataFrame 的 duplicated()方法傳回一個 boolean型態的 Series，表示各row是否重複
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [142]:
# drop_duplicates()方法 傳回移除重複項目之後的結果
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [143]:
data['k3'] = range(7)
data

,k1,k2,k3
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6


In [144]:
# drop_duplicates()預設會對所有的columns來判斷是否有重複的 rows
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6    False
dtype: bool

In [145]:
# 也可以針對指定的columns來判斷是否有重複的 rows
data.duplicated(['k1'])

0    False
1     True
2     True
3    False
4     True
5     True
6     True
dtype: bool

In [146]:
# duplicated, drop_duplicates 預設保留第一個出現的值組合
# 設定參數 keep = 'last'，則會改為保留最後一個出現的值組合
data.duplicated(['k1'], keep = 'last')

0     True
1     True
2    False
3     True
4     True
5     True
6    False
dtype: bool

### 利用函數或映射進行數據轉換###

In [147]:
data = DataFrame({'food':['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'], 
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [148]:
meat_to_animal = {'bacon': 'pig', 
                  'pulled pork': 'pig', 
                  'pastrami': 'cow', 
                  'corned beef': 'cow', 
                  'honey ham': 'pig', 
                  'nova lox': 'salmon'}

In [149]:
# Series 的 map()方法，可以將元素map給特定的 字典或函數 來進行轉換
# 需先規整大小寫，也是透過 map 對每個元素做 str.lower的操作
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [150]:
# 也可以透過 lambda來做
data['animal'] = data['food'].map(lambda x: meat_to_animal[x.lower()])
data
# 使用 map()是實現元素級清理與轉換的便捷方式

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### 替換值###

In [151]:
data = Series([1., -999., 2., -999., -1000., 3.,])
data

0       1
1    -999
2       2
3    -999
4   -1000
5       3
dtype: float64

In [152]:
# 用 replace()方法來置換數值
data.replace(-999, np.nan)

0       1
1     NaN
2       2
3     NaN
4   -1000
5       3
dtype: float64

In [153]:
# 一次置換多個值。要被替換的包裝在一個list中
data.replace([-999, -1000], np.nan)

0     1
1   NaN
2     2
3   NaN
4   NaN
5     3
dtype: float64

In [154]:
# 對不同值 進行不同的替換
# 要被替換的放在第一個 list, 替換者放在第二個 list，要匹配
data.replace([-999, -1000], [np.nan, 0])

0     1
1   NaN
2     2
3   NaN
4     0
5     3
dtype: float64

In [155]:
# 替代關係用 dict表達會比較清楚
data.replace({-999: np.nan, -1000: 0})

0     1
1   NaN
2     2
3   NaN
4     0
5     3
dtype: float64

### 重新命名軸索引###

軸標籤也可以進行轉換，或者就地修改

In [156]:
data = DataFrame(np.arange(12).reshape((3, 4)), 
                 index = pd.Index(['Ohio', 'Colorado', 'New York'], name = 'state'), 
                 columns = pd.Index(['one', 'two', 'three', 'four'], name = 'quarter'))
data

quarter,one,two,three,four
state,,,,
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [157]:
# Index 也有一個 map()方法，可以傳回一個新的 Index物件
data.index = data.index.map(str.upper)
data

quarter,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [158]:
# rename()方法會傳回一個數據集的轉換版本，而不是修改原來的數據
# 使用 index, columns 指定的函式 來修改軸標籤
data.rename(index = str.title, columns = str.upper)

quarter,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [159]:
# rename 可以結合字典型物件，實現對部分軸標籤的更新
data.rename(index = {'OHIO': 'INDIANA'}, columns = {'three': 'peekaboo'})

quarter,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [160]:
data

quarter,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [161]:
# 如果希望就地修改原有的數據集，使需要在 rename()方法中設定參數 inplace = True
data.rename(index = {'OHIO': 'INDIANA'}, columns = {'three': 'peekaboo'}, inplace = True)
data

quarter,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


### 離散化和面元(bin)劃分###

In [169]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
# 劃分為幾個 面元(bin)，可以使用 cut()函式
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, object): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [170]:
# 傳回的是一個 Categorical 物件
type(cats)

pandas.core.categorical.Categorical

In [171]:
# Categorical物件的 codes屬性，標示原始元素所屬的分組編號
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [177]:
# Categorical物件的 categories屬性，標示各個分組的 範圍 (以 str表達)
cats.categories
# categories屬性是一個 Index 物件

Index(['(18, 25]', '(25, 35]', '(35, 60]', '(60, 100]'], dtype='object')

In [180]:
# 統計各個區間的數值數量
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [185]:
# 區間: 圓括號代表開端(不包括)，方括號代表閉端(包括)
# 哪一邊是閉端 可以透過參數 right = True 來設定，預設是 True
cats = pd.cut(ages, bins, right = False)
cats

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, object): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [186]:
cats.codes

array([0, 0, 1, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [187]:
pd.value_counts(cats)

[25, 35)     4
[18, 25)     4
[35, 60)     3
[60, 100)    1
dtype: int64

In [198]:
# 使用 labels 參數，可以對 bin 設定各個區間的 label 標籤
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
cats = pd.cut(ages, bins, labels = group_names, right = True)
cats

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [203]:
# 如果向cut傳入的是bin的數量 而不是確切的 bin邊界，則 cut()會根據最小值、最大值平均分配bin大小
data = np.random.rand(20)
cats = pd.cut(data, 4, precision = 2)
cats

[(0.5, 0.73], (0.27, 0.5], (0.04, 0.27], (0.73, 0.96], (0.27, 0.5], ..., (0.5, 0.73], (0.04, 0.27], (0.73, 0.96], (0.5, 0.73], (0.73, 0.96]]
Length: 20
Categories (4, object): [(0.04, 0.27] < (0.27, 0.5] < (0.5, 0.73] < (0.73, 0.96]]

In [204]:
pd.value_counts(cats)

(0.73, 0.96]    6
(0.04, 0.27]    6
(0.5, 0.73]     5
(0.27, 0.5]     3
dtype: int64

In [211]:
# qcut() 會根據 樣本分位數 對數據進行 bin劃分，因此可以得到大小基本相等的bin
data = np.random.rand(997)
cats = pd.qcut(data, 4)
cats

[(0.247, 0.506], (0.506, 0.748], (0.506, 0.748], [0.000396, 0.247], [0.000396, 0.247], ..., [0.000396, 0.247], (0.247, 0.506], [0.000396, 0.247], (0.748, 1], (0.748, 1]]
Length: 997
Categories (4, object): [[0.000396, 0.247] < (0.247, 0.506] < (0.506, 0.748] < (0.748, 1]]

In [212]:
pd.value_counts(cats)
# 每個 category 的元素數量基本上相當

[0.000396, 0.247]    250
(0.748, 1]           249
(0.506, 0.748]       249
(0.247, 0.506]       249
dtype: int64

In [213]:
# 可以自定義分位數
cats = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
cats

[(0.104, 0.506], (0.506, 0.897], (0.506, 0.897], [0.000396, 0.104], (0.104, 0.506], ..., [0.000396, 0.104], (0.104, 0.506], (0.104, 0.506], (0.897, 1], (0.506, 0.897]]
Length: 997
Categories (4, object): [[0.000396, 0.104] < (0.104, 0.506] < (0.506, 0.897] < (0.897, 1]]

In [214]:
pd.value_counts(cats)

(0.104, 0.506]       399
(0.506, 0.897]       398
(0.897, 1]           100
[0.000396, 0.104]    100
dtype: int64

### 檢測和過濾異常值(outlier)###

In [220]:
# 常態分布陣列
np.random.seed(12345)
df = DataFrame(np.random.randn(1000, 4))
df.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [223]:
# 找出某列中，絕對值大於3的數值
col = df[2]
col[np.abs(col) > 3]

5      3.248944
102    3.176873
324    3.260383
499   -3.056990
586   -3.184377
Name: 2, dtype: float64

In [231]:
# 找出所有 含有絕對值大於3的數值 的row，可以運用 any()
df[(np.abs(df) > 3).any(axis = 1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [236]:
# 將陣列數值限制在 +-3之間
gt3 = (np.abs(df) > 3)
df[gt3] = np.sign(df) * 3
df.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


### 排列(permuting)和隨機採樣###

In [243]:
df = DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [246]:
# np.random.permutation()可以對Series或DataFrame的列作 隨機重新排列的動作
# permutation()會產生一個表示新順序的整數數組
sampler = np.random.permutation(len(df))
sampler

array([1, 4, 0, 2, 3])

In [247]:
# 使用 take()方法 使用重排之後的順序，由原本的數組中建立新的數組
df.take(sampler)

,0,1,2,3
1,4,5,6,7
4,16,17,18,19
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15


In [249]:
# 非替換式採樣，從母體中隨機採取樣本
# 使用 permutation 隨機取樣，並切取所需要的樣本大小
df.take(np.random.permutation(len(df))[:3])

,0,1,2,3
0,0,1,2,3
3,12,13,14,15
2,8,9,10,11


In [7]:
# 要通過替換的方式產生樣本，最快的方式是通過 np.random.randint()得到一組隨機整數
bag = np.array([5, 7, -1, 6, 4]) # 原始數據

# 產生 10的索引，索引值介於 0~len(原始數組)
sampler = np.random.randint(0, len(bag), size = 10)
sampler

array([4, 1, 3, 4, 2, 4, 2, 3, 3, 4])

In [8]:
# 依據 sampler中的取樣索引值，由原來的數組中 取10個樣品
draws = bag.take(sampler)
draws

array([ 4,  7,  6,  4, -1,  4, -1,  6,  6,  4])

### 計算指標/啞變量###

另一種常見的數據轉換方式: 將 分類變量(categorical variable) 轉換為 啞變量矩陣(dummy matrix) 或 指標矩陣(indicator matrix)

In [12]:
df = DataFrame({'key': list('bbacab'), 
                'data1': range(6)})
df
# DataFrame中的某一列中含有 k個不同的值，則可以派生出一個 k列矩陣或DataFrame(其值全部為 0或1)

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [13]:
# df['key'] 中有 a, b, c 3個不同的值，可以派生出一個 3列的矩陣或DataFrame
# 用 get_dummies可以取得一個 dummy matrix
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [14]:
# 有時需要對DataFrame的列索引加上一個前綴，以便和其他的數據進行合併
# get_dummies()方法的 prefix 餐數可以達到此功能
dummies = pd.get_dummies(df['key'], prefix = 'key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [19]:
df[['data1']]

,data1
0,0
1,1
2,2
3,3
4,4
5,5


In [21]:
df_with_dummies = df[['data1']].join(dummies)
df_with_dummies

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


#### DataFrame中的某行同屬於多個分類####

In [3]:
# 讀取 MovieLens 資料
# http://grouplens.org/datasets/movielens/
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('D:\\a 電腦類書籍 CD\\Python\\python for data analysis\\pydata-book\\ch07\\ml-1m\\users.dat', sep = '::', header = None, names = unames)
rnames = ['user_id', 'move-id', 'rating', 'timestamp']
ratings = pd.read_table('D:\\a 電腦類書籍 CD\\Python\\python for data analysis\\pydata-book\\ch07\\ml-1m\\ratings.dat', sep = '::', header = None, names = rnames)
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('D:\\a 電腦類書籍 CD\\Python\\python for data analysis\\pydata-book\\ch07\\ml-1m\\movies.dat', sep = '::', header = None, names = mnames)


D:\Python\lib\site-packages\IPython\kernel\__main__.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
D:\Python\lib\site-packages\IPython\kernel\__main__.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
D:\Python\lib\site-packages\IPython\kernel\__main__.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.


In [4]:
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [51]:
# 由 genres中抽出不同的 genre值
# 先建立一個 generator
genre_iter = (set(x.split('|')) for x in movies['genres'])

# 展開 generator，並使用 set.union()來做聯集
genres = sorted(set.union(*genre_iter))
genres

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [52]:
# 創建一個空的 關聯表
dummies = DataFrame(np.zeros((len(movies), len(genres))), columns = genres)
dummies

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
# 依據 每部電影的 genres資料，填寫matrix
for i, gen in enumerate(movies['genres']):
    dummies.ix[i, gen.split('|')] = 1

dummies

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
6,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
7,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [54]:
# 把 movies的基本資料與 類別matrix join起來，同時標示 prefix
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic

,movie_id,title,genres,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Genre_Fantasy,Genre_Film-Noir,Genre_Horror,Genre_Musical,Genre_Mystery,Genre_Romance,Genre_Sci-Fi,Genre_Thriller,Genre_War,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,Heat (1995),Action|Crime|Thriller,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
6,7,Sabrina (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
7,8,Tom and Huck (1995),Adventure|Children's,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9,Sudden Death (1995),Action,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,GoldenEye (1995),Action|Adventure|Thriller,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [55]:
movies_windic.ix[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       

In [57]:
# 可以結合 get_dummies 和如 cut()之類的離散化函數
values = np.random.rand(10)
values

array([ 0.97147589,  0.78917385,  0.66358729,  0.25514239,  0.88972394,
        0.37456403,  0.04182371,  0.89237963,  0.17816452,  0.42184859])

In [59]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

# 切分分組
pd.cut(values, bins)

[(0.8, 1], (0.6, 0.8], (0.6, 0.8], (0.2, 0.4], (0.8, 1], (0.2, 0.4], (0, 0.2], (0.8, 1], (0, 0.2], (0.4, 0.6]]
Categories (5, object): [(0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1]]

In [60]:
# 可以把上面分組的結果，展開成為 dummy matrix
pd.get_dummies(pd.cut(values, bins))

,"(0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1]"
0,0,0,0,0,1
1,0,0,0,1,0
2,0,0,0,1,0
3,0,1,0,0,0
4,0,0,0,0,1
5,0,1,0,0,0
6,1,0,0,0,0
7,0,0,0,0,1
8,1,0,0,0,0
9,0,0,1,0,0


## 字符串操作##

### 字符串對象方法###

In [68]:
# 分隔字符可以用split來拆分
s = 'a, b,    guido'
names = s.split(',')
names

['a', ' b', '    guido']

In [70]:
# split 經常要結合 strip() 來做清理空白
names = [name.strip() for name in names ]
names

['a', 'b', 'guido']

In [71]:
# 用 join 可以連接字串
names = '::'.join(names)
names

'a::b::guido'

In [72]:
# 用 in 來檢測字串是否存在
'guido' in s

True

In [75]:
# index()傳回特定字串首次出現的位置，由左邊開始尋找，如果找不到則傳回 ValueError
s.index('guido')

9

In [76]:
# find() 傳回特定字串首次出現的位置，由左邊開始尋找，如果找不到則傳回 -1
s.find('guido')

9

In [77]:
s.find('xxxx')

-1

In [78]:
# rfind()由右往左尋找
s.rfind('guido')

9

In [80]:
# count()返回特定字串出現的次數
s.count(',')

2

In [81]:
# replace() 用來取代字串
s.replace(',', '::')

'a:: b::    guido'

### 正則表達式###

In [2]:
import re

In [82]:
text = 'foo    bar\t    baz    \tqux'

# '\s+' 代表各種空白字元
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [84]:
# findall()列出所以匹配到的字串
regex.findall(text)

['    ', '\t    ', '    \t']

In [27]:
text = """
Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

In [7]:
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
regex = re.compile(pattern, flags = re.IGNORECASE)

In [100]:
# findall() 會找出所有相符合者
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [101]:
# search()只會傳回第一個相符合者 對應的 SRE_Match 物件
se = regex.search(text)
type(se)

_sre.SRE_Match

In [102]:
# SRE_Match 有 start(), end()方法，可以取得 匹配字串的 起訖位置
text[se.start():se.end()]

'dave@google.com'

In [108]:
# match()只會匹配字串的首部，只匹配出現在字串開頭的模式
regex.match(text)

In [107]:
# sub()方法會將匹配到的模式替換為指定的字串並返回新字串
regex.sub('REDACTED', text)

'\nDave REDACTED\nSteve REDACTED\nRob REDACTED\nRyan REDACTED\n'

In [106]:
print(regex.sub('REDACTED', text))


Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [16]:
# 想將各個地址分成3個部分，只需將待分段的各部分用圓括號包起來
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags = re.IGNORECASE)

In [17]:
# 使用 match()方法來匹配
m = regex.match('wesm@bright.net')
m

<_sre.SRE_Match object; span=(0, 15), match='wesm@bright.net'>

In [18]:
# SRE_Match物件的 group()方法會傳回分段之後的元組
m.groups()

('wesm', 'bright', 'net')

In [24]:
# 經過正則表達式匹配之後，findall()會傳回分段的元組列表
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [23]:
# sub()還能透過如 \1, \2之類的特殊符號訪問各匹配項中的分組
print (regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))


Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



In [16]:
# 為各個匹配分組加上一個名稱
pattern = r"""
(?P<username>[A-Z0-9._%+-]+)
@
(?P<domain>[A-Z0-9.-]+)
\.
(?P<suffix>[A-Z]{2,4})
"""
regex = re.compile(pattern, flags = re.IGNORECASE | re.VERBOSE)

In [31]:
m = regex.match('wesm@bright.net')
m

<_sre.SRE_Match object; span=(0, 15), match='wesm@bright.net'>

In [32]:
# 由 match() 的 groupdict()方法，可以傳回一個分段名稱的字典
m.groupdict()

{'domain': 'bright', 'suffix': 'net', 'username': 'wesm'}

### pandas中向量化的字串函數###

In [3]:
data = Series({'Dave': 'dave@google.com', 
        'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 
        'Wes': np.nan})
data

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

In [8]:
# 透過 Series 的 str屬性 可以訪問一些字串的方法
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

In [10]:
# 是一個 StringMethods物件，之下掛了很多字串方法
data.str

In [17]:
# .str 之下也掛有 reqular expression 的一些方法
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

# reg 的 findall()方法
data.str.findall(pattern, flags = re.IGNORECASE)

Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes                        NaN
dtype: object

In [18]:
# reg 的 match()方法
matchs = data.str.match(pattern, flags = re.IGNORECASE)
matchs

D:\Python\lib\site-packages\IPython\kernel\__main__.py:1: FutureWarning: In future versions of pandas, match will change to always return a bool indexer.
  if __name__ == '__main__':


Dave     (dave, google, com)
Rob        (rob, gmail, com)
Steve    (steve, gmail, com)
Wes                      NaN
dtype: object

In [20]:
# 提取匹配結果中 索引為 1 的元素
matchs.str.get(1)

Dave     google
Rob       gmail
Steve     gmail
Wes         NaN
dtype: object

In [21]:
matchs.str[0]

Dave      dave
Rob        rob
Steve    steve
Wes        NaN
dtype: object

In [28]:
# 對字串進行子串擷取
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object

## 範例: USDA食品數據庫##

In [2]:
import json

db = json.load(open('D:\\Temp\\pydata-book\\ch07\\foods-2011-10-03.json'))
len(db)

6636

In [6]:
db[0].keys()

dict_keys(['id', 'group', 'description', 'nutrients', 'portions', 'tags', 'manufacturer'])

In [8]:
db[0]['nutrients'][0]

{'description': 'Protein',
 'group': 'Composition',
 'units': 'g',
 'value': 25.18}

In [13]:
nutrients = DataFrame(db[0]['nutrients'])
nutrients[:7]

,description,group,units,value
0,Protein,Composition,g,25.18
1,Total lipid (fat),Composition,g,29.20
2,"Carbohydrate, by difference",Composition,g,3.06
3,Ash,Other,g,3.28
4,Energy,Energy,kcal,376.00
5,Water,Composition,g,39.28
6,Energy,Energy,kJ,1573.00


In [14]:
type(db)

list

In [20]:
# 從字典列表中，只抽取一部分字段
keys = ['description', 'group', 'id', 'manufacturer']
info = DataFrame(db, columns = keys)
info[:20]

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,
5,"Cheese, mozzarella, part skim milk, low moisture",Dairy and Egg Products,1029,
6,"Cheese, romano",Dairy and Egg Products,1038,
7,"Cheese, roquefort",Dairy and Egg Products,1039,
8,"Cheese spread, pasteurized process, american, ...",Dairy and Egg Products,1048,
9,"Cream, fluid, half and half",Dairy and Egg Products,1049,


In [23]:
# 統計 group 值出現的次數，數據的分布狀況
pd.value_counts(info['group'])

Vegetables and Vegetable Products    812
Beef Products                        618
Baked Products                       496
Breakfast Cereals                    403
Legumes and Legume Products          365
Fast Foods                           365
Lamb, Veal, and Game Products        345
Sweets                               341
Pork Products                        328
Fruits and Fruit Juices              328
Beverages                            278
Soups, Sauces, and Gravies           275
Finfish and Shellfish Products       255
Baby Foods                           209
Cereal Grains and Pasta              183
Ethnic Foods                         165
Snacks                               162
Nut and Seed Products                128
Poultry Products                     116
Sausages and Luncheon Meats          111
Dairy and Egg Products               107
Fats and Oils                         97
Meals, Entrees, and Sidedishes        57
Restaurant Foods                      51
Spices and Herbs

In [30]:
# 將所有食物的營養成分整合到一個大表中
nutrients = []
for rec in db:
    fnuts = DataFrame(rec['nutrients'])
    fnuts['id'] = rec['id']
    nutrients.append(fnuts)
    
nutrients = pd.concat(nutrients, ignore_index = True)
nutrients[:7]

,description,group,units,value,id
0,Protein,Composition,g,25.18,1008
1,Total lipid (fat),Composition,g,29.20,1008
2,"Carbohydrate, by difference",Composition,g,3.06,1008
3,Ash,Other,g,3.28,1008
4,Energy,Energy,kcal,376.00,1008
5,Water,Composition,g,39.28,1008
6,Energy,Energy,kJ,1573.00,1008


In [31]:
nutrients.duplicated().sum()

14223

In [32]:
nutrients = nutrients.drop_duplicates()
nutrients[:7]

,description,group,units,value,id
0,Protein,Composition,g,25.18,1008
1,Total lipid (fat),Composition,g,29.20,1008
2,"Carbohydrate, by difference",Composition,g,3.06,1008
3,Ash,Other,g,3.28,1008
4,Energy,Energy,kcal,376.00,1008
5,Water,Composition,g,39.28,1008
6,Energy,Energy,kJ,1573.00,1008


In [34]:
col_mapping = {'description': 'food', 
               'group': 'fgroup'}
info = info.rename(columns = col_mapping, copy = False)
info[:7]

,food,fgroup,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,
5,"Cheese, mozzarella, part skim milk, low moisture",Dairy and Egg Products,1029,
6,"Cheese, romano",Dairy and Egg Products,1038,


In [35]:
col_mapping = {'description': 'nutrient', 
               'group': 'nutgroup'}
nutrients = nutrients.rename(columns = col_mapping, copy = False)
nutrients[:7]

,nutrient,nutgroup,units,value,id
0,Protein,Composition,g,25.18,1008
1,Total lipid (fat),Composition,g,29.20,1008
2,"Carbohydrate, by difference",Composition,g,3.06,1008
3,Ash,Other,g,3.28,1008
4,Energy,Energy,kcal,376.00,1008
5,Water,Composition,g,39.28,1008
6,Energy,Energy,kJ,1573.00,1008


In [40]:
ndata = pd.merge(info, nutrients, on = 'id', how = 'outer')
ndata = ndata.reindex(columns = ['id', 'food', 'fgroup', 'manufacturer', 'nutgroup', 'nutrient', 'unit', 'value'])
ndata[:7]

,id,food,fgroup,manufacturer,nutgroup,nutrient,unit,value
0,1008,"Cheese, caraway",Dairy and Egg Products,,Composition,Protein,NaN,25.18
1,1008,"Cheese, caraway",Dairy and Egg Products,,Composition,Total lipid (fat),NaN,29.20
2,1008,"Cheese, caraway",Dairy and Egg Products,,Composition,"Carbohydrate, by difference",NaN,3.06
3,1008,"Cheese, caraway",Dairy and Egg Products,,Other,Ash,NaN,3.28
4,1008,"Cheese, caraway",Dairy and Egg Products,,Energy,Energy,NaN,376.00
5,1008,"Cheese, caraway",Dairy and Egg Products,,Composition,Water,NaN,39.28
6,1008,"Cheese, caraway",Dairy and Egg Products,,Energy,Energy,NaN,1573.00


In [41]:
ndata.ix[10000]

id                                                    3197
food            Babyfood, cereal, with egg yolks, strained
fgroup                                          Baby Foods
manufacturer                                              
nutgroup                                       Amino Acids
nutrient                                          Arginine
unit                                                   NaN
value                                                0.104
Name: 10000, dtype: object

In [44]:
result = ndata.groupby(['nutrient', 'fgroup'])['value'].quantile(0.5)
result['Zinc, Zn'].sort_values().plot(kind = 'barh')
by_nutrient = ndata.groupby(['nutgroup', 'nutrient'])
get_maximum = lambda x: x.xs(x.value.idxmax())
get_minimum = lambda x: x.xs(x.value.idxmin())
max_foods = by_nutrient.apply(get_maximum)[['value', 'food']]
max_foods.food = max_foods.food.str[:50]

In [46]:
max_foods.ix['Amino Acids']['food']

nutrient
Alanine                           Gelatins, dry powder, unsweetened
Arginine                               Seeds, sesame flour, low-fat
Aspartic acid                                   Soy protein isolate
Cystine                Seeds, cottonseed flour, low fat (glandless)
Glutamic acid                                   Soy protein isolate
Glycine                           Gelatins, dry powder, unsweetened
Histidine                Whale, beluga, meat, dried (Alaska Native)
Hydroxyproline    KENTUCKY FRIED CHICKEN, Fried Chicken, ORIGINA...
Isoleucine        Soy protein isolate, PROTEIN TECHNOLOGIES INTE...
Leucine           Soy protein isolate, PROTEIN TECHNOLOGIES INTE...
Lysine            Seal, bearded (Oogruk), meat, dried (Alaska Na...
Methionine                    Fish, cod, Atlantic, dried and salted
Phenylalanine     Soy protein isolate, PROTEIN TECHNOLOGIES INTE...
Proline                           Gelatins, dry powder, unsweetened
Serine            Soy protein isolate, 